In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report 
from sklearn.linear_model import LogisticRegression
import pickle

#load dataset
df = pd.read_csv('US-pumpkins.csv')

cols = ['City Name','Package','Variety', 'Origin','Item Size', 'Color']
   
df = df.loc[:, cols]
df.dropna(inplace=True)

catergorical_features = ['City Name', 'Package', 'Variety', 'Origin']
catergorical_encoder = OneHotEncoder(sparse_output=False)


item_size_categories = [['sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo']]
Ordinal_feature = ['Item Size']
Ordinal_encoder = OrdinalEncoder(categories=item_size_categories)

#combine categorical and ordinal features(encoded)
ct = ColumnTransformer( transformers=
    [('categorical', catergorical_encoder, catergorical_features),
     ('ordinal', Ordinal_encoder, Ordinal_feature)],
    remainder='drop'
)

#fit
ct.set_output(transform='pandas')
encoded_features = ct.fit_transform(df)

#label encoding

label_encoder = LabelEncoder()
encoded_label = label_encoder.fit_transform(df['Color'])

#new df
encoded_df = encoded_features.assign(Color=encoded_label)

x = encoded_df[encoded_df.columns.difference(['Color'])]
y = encoded_df['Color']

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

#training the model

model = LogisticRegression()
model.fit(x_train,y_train)

model_filename = 'ufo-model.pkl'
pickle.dump(model, open(model_filename,'wb'))

model = pickle.load(open('ufo-model.pkl','rb'))